# Stochastic Parameters Modelling

## Market Data

In [ ]:
from fredapi import Fred
import pandas as pd

# Initialize FRED (get a free API key from FRED)
fred = Fred(api_key=API_KEY)

# PPI: Hot Rolled Steel Bars, Plates, and Structural Shapes
P = fred.get_series("WPU101704").to_frame(name="P_steel_ppi")

# PPI: Iron and Steel Scrap
C = fred.get_series("WPU1012").to_frame(name="C_scrap_ppi")

# IPG3311A2S: Industrial Production Index: Steel Products
D = fred.get_series("IPG3311A2S").to_frame(name="D_steel_ip")

P.index = pd.to_datetime(P.index)
C.index = pd.to_datetime(C.index)
D.index = pd.to_datetime(D.index)

P = P.resample("MS").mean()
C = C.resample("MS").mean()
D = D.resample("MS").mean()

data = pd.concat([P, C, D], axis=1).dropna()

data.plot(title="Finished Steel Product Price vs. Scrap Price vs. Steel Industrial Production", ylabel="Index (1982=100)")
data

In [ ]:
# Rule of thumb for number of data points
# ~ 10 * k^2 * p
# k = number of variables
# p = number of lags in VAR

k = 3 
p = 2
n_data_points = 10 * k**2 * p

data = data[-n_data_points:]
data.plot(title="Finished Steel Product Price vs. Scrap Price vs. Steel Industrial Production", ylabel="Index (1982=100)")
data

## VAR Model

Δlog Returns & Distributions

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

# Δlog(x_t) = log(x_t) - log(x_{t-1})  ~ monthly % change
Δlog = np.log(data[["D_steel_ip", "P_steel_ppi", "C_scrap_ppi"]]).diff().dropna()

Δlog.plot(title="Log-Returns of Steel Demand, Price, and Scrap Cost")
Δlog.hist(bins=30, figsize=(10, 6))

# NOTE: Volatility Ranking
# It is usually expected to have: std(Δlog D) << std(Δlog P) < std(Δlog C)
# However since the dataset are index data which are smoothed, we may not see this pattern clearly

# NOTE: Kurtosis and Skewness
# Heavier tails (kurtosis > 3) may be observed due to economic shocks of 2008 and COVID-19 in 2020
print(Δlog.agg(["mean", "std", "skew", "kurtosis"]))

Order Selection

In [ ]:
# Fit VAR model
model = VAR(Δlog)

# Pick lag via BIC/AIC (recommended)
lag_sel = model.select_order(maxlags=12)   # monthly -> try up to 12

# NOTE: EXPECTED LAG ORDER IS SMALL
# It is expected for steel markets to have p = 1 or 2 months
# since production reacts fast to price changes
p = lag_sel.selected_orders["bic"]
print("Selected lag order (BIC):", p)
lag_sel.summary()

Model Fit & Testing

In [ ]:
# Fit VAR(p)
var_res = model.fit(p)

# Check Correlation of Residuals
# NOTE: EXPECTED BEHAVIOR
# 1. Price vs Demand: low correlation is expected since price changes should not instantaneously affect demand
# 2. Cost vs Price: cost partially affects immediate price changes and some are delayed
# 3. Cost vs Demand: higher demand lead to higher scrap prices due to higher steel production 
# (*) some may differ since index data are considered
print("Residual Correlation Matrix:")
print(var_res.resid.corr())


# Impulse Response Functions
# NOTE: Orthogonalized vs Non-Orthogonalized
# Orthogonalized IRF assumes shocks are uncorrelated (Cholesky decomposition)
# Non-Orthogonalized IRF allows correlated shocks (more realistic in economics)
from matplotlib import pyplot as plt
irf = var_res.irf(12)  # 12-month horizon
irf.plot(orth=False)
plt.show()
irf.plot_cum_effects(orth=False)
plt.show()

# Simulate Future Scenarios
import numpy as np
S, T = 5000, 24

sim_list = []
for _ in range(S):
    path = var_res.simulate_var(steps=T)
    sim_list.append(path)

sim = np.array(sim_list)

print(f"Simulation shape: {sim.shape}")  # Should be (5000, 24, 3)

hist_corr = Δlog.corr()
sim_flat = sim.reshape(-1, sim.shape[2])
sim_corr = pd.DataFrame(sim_flat, columns=Δlog.columns).corr()

print("\nHistorical correlation:\n", hist_corr)
print("\nSimulated correlation:\n", sim_corr)

## Scenario Generation

In [ ]:
import numpy as np
import pandas as pd

# -----------------------------
# Settings
# -----------------------------
HORIZON_MONTHS = 12
N_SCENARIOS = 50_000
SEED = 42

# Anchors (Option 2) - pick a month where you know (or assume) real prices
t0 = data.index.max()     # must exist in df.index
P0_real = 800.0       # €/ton (example finished steel price at t0)
C0_real = 400.0       # €/ton (example scrap price at t0)

# Demand scaling (optional):
# If you want demand in tons instead of an index, set one anchor:
D0_real = 50_0000.0    # tons at t0 (example)
use_demand_scaling_to_tons = True  # set True if you want tons

# -----------------------------
# Helpers
# -----------------------------
def simulate_var_returns(var_res, steps: int, n_scenarios: int, seed: int = 0) -> np.ndarray:
    """
    Simulates VAR returns using Gaussian innovations with covariance sigma_u.
    Output shape: (S, T, k)
    """
    rng = np.random.default_rng(seed)
    k = var_res.neqs
    Sigma = var_res.sigma_u.values
    chol = np.linalg.cholesky(Sigma)

    # last p observations of endogenous returns
    p = var_res.k_ar
    y_hist0 = var_res.endog[-p:].copy()

    sims = np.zeros((n_scenarios, steps, k))
    for s in range(n_scenarios):
        y_hist = y_hist0.copy()
        for t in range(steps):
            eps = chol @ rng.standard_normal(k)
            yhat = var_res.forecast(y_hist, steps=1)[0]
            ynew = yhat + eps
            sims[s, t, :] = ynew

            if p > 1:
                y_hist = np.vstack([y_hist[1:], ynew])
            else:
                y_hist = np.array([ynew])
    return sims

def reconstruct_from_returns(last_level: float, returns: np.ndarray) -> np.ndarray:
    """
    Given last observed level and a sequence of Δlog returns,
    reconstruct levels: level_t = last_level * exp(cumsum(returns))
    returns: shape (T,)
    """
    return last_level * np.exp(np.cumsum(returns))

def anchor_index_to_real(ppi_path: np.ndarray, ppi_t0: float, real_t0: float) -> np.ndarray:
    """
    Convert index path to real €/ton via anchoring:
    price_t = real_t0 * (ppi_t / ppi_t0)
    """
    return real_t0 * (ppi_path / ppi_t0)

# -----------------------------
# Columns order (must match VAR endog order)
# -----------------------------
cols = ["D_steel_ip", "P_steel_ppi", "C_scrap_ppi"]

# Returns data used for VAR
df_ret = np.log(data[cols]).diff().dropna()

# -----------------------------
# 1) Simulate return scenarios from VAR(2)
# -----------------------------
sim_ret = simulate_var_returns(var_res, steps=HORIZON_MONTHS, n_scenarios=N_SCENARIOS, seed=SEED)
# sim_ret shape: (S, T, k)

# Map indices
k_names = list(df_ret.columns)
iD = k_names.index("D_steel_ip")
iP = k_names.index("P_steel_ppi")
iC = k_names.index("C_scrap_ppi")

# -----------------------------
# 2) Decide simulation start date + last observed levels
# -----------------------------
start_date = (data.index.max() + pd.offsets.MonthBegin(1)).normalize()
dates = pd.date_range(start=start_date, periods=HORIZON_MONTHS, freq="MS")

last_date = data.index.max()
D_last = float(data.loc[last_date, "D_steel_ip"])
P_last = float(data.loc[last_date, "P_steel_ppi"])
C_last = float(data.loc[last_date, "C_scrap_ppi"])

# -----------------------------
# 3) Reconstruct index level paths from Δlog returns
# -----------------------------
D_paths_idx = np.zeros((HORIZON_MONTHS, N_SCENARIOS))
P_paths_ppi = np.zeros((HORIZON_MONTHS, N_SCENARIOS))
C_paths_ppi = np.zeros((HORIZON_MONTHS, N_SCENARIOS))

for s in range(N_SCENARIOS):
    D_paths_idx[:, s] = reconstruct_from_returns(D_last, sim_ret[s, :, iD])
    P_paths_ppi[:, s] = reconstruct_from_returns(P_last, sim_ret[s, :, iP])
    C_paths_ppi[:, s] = reconstruct_from_returns(C_last, sim_ret[s, :, iC])

D_paths_idx = pd.DataFrame(D_paths_idx, index=dates, columns=[f"s{s}" for s in range(N_SCENARIOS)])
P_paths_ppi = pd.DataFrame(P_paths_ppi, index=dates, columns=[f"s{s}" for s in range(N_SCENARIOS)])
C_paths_ppi = pd.DataFrame(C_paths_ppi, index=dates, columns=[f"s{s}" for s in range(N_SCENARIOS)])

# -----------------------------
# 4) Anchor PPIs to real €/ton (Option 2)
# -----------------------------
t0_ts = pd.Timestamp(t0)
if t0_ts not in data.index:
    raise ValueError(f"t0={t0} not found in data.index. Choose a YYYY-MM-01 that exists.")

P_ppi_t0 = float(data.loc[t0_ts, "P_steel_ppi"])
C_ppi_t0 = float(data.loc[t0_ts, "C_scrap_ppi"])
P_paths_eur = P_paths_ppi.apply(lambda col: anchor_index_to_real(col.values, P_ppi_t0, P0_real), axis=0)
C_paths_eur = C_paths_ppi.apply(lambda col: anchor_index_to_real(col.values, C_ppi_t0, C0_real), axis=0)

P_paths_eur = pd.DataFrame(P_paths_eur.values, index=dates, columns=P_paths_ppi.columns)
C_paths_eur = pd.DataFrame(C_paths_eur.values, index=dates, columns=C_paths_ppi.columns)

# -----------------------------
# 5) Demand: keep as index OR scale to tons using an anchor
# -----------------------------
if use_demand_scaling_to_tons:
    D_idx_t0 = float(data.loc[t0_ts, "D_steel_ip"])
    D_paths_tons = D0_real * (D_paths_idx / D_idx_t0)
    D_final = D_paths_tons
else:
    D_final = D_paths_idx  # index units (fine; you can later scale)

# -----------------------------
# 6) Export scenarios in long form for the SP
# -----------------------------
def wide_to_long(wide: pd.DataFrame, value_name: str) -> pd.DataFrame:
    out = wide.stack().rename(value_name).reset_index()
    out = out.rename(columns={"level_0": "Date", "level_1": "Scenario"})
    return out

scen_D = wide_to_long(D_final, cols[0])
scen_P = wide_to_long(P_paths_eur, cols[1])
scen_C = wide_to_long(C_paths_eur, cols[2])

scenarios = scen_D.merge(scen_P, on=["Date", "Scenario"]).merge(scen_C, on=["Date", "Scenario"])
scenarios

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create comprehensive scenario projection plots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Number of scenarios to plot (for clarity, plot subset)
n_plot = min(50, N_SCENARIOS)

# Generate date range for plotting
start_date = (data.index.max() + pd.offsets.MonthBegin(1)).normalize()
future_dates = pd.date_range(start=start_date, periods=HORIZON_MONTHS, freq="MS")

# Combine historical and future dates for context
hist_months = 32  # Show last 32 months of history
hist_dates = data.index[-hist_months:]

# Plot 1: Demand Index Scenarios
ax = axes[0, 0]
# Historical data
ax.plot(hist_dates, data.loc[hist_dates, "D_steel_ip"], 'k-', linewidth=2, label='Historical')
# Future scenarios
for s in range(n_plot):
    ax.plot(future_dates, D_paths_idx.iloc[:, s], alpha=0.3, linewidth=0.8, color='steelblue')
# Mean projection
ax.plot(future_dates, D_paths_idx.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
# Confidence bands (10th-90th percentile)
ax.fill_between(future_dates, 
                D_paths_idx.quantile(0.1, axis=1), 
                D_paths_idx.quantile(0.9, axis=1),
                alpha=0.2, color='red', label='10th-90th Percentile')
ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_title('Demand Index Projection\n(Industrial Production)', fontsize=12, fontweight='bold')
ax.set_ylabel('Index (1982=100)', fontsize=10)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Price PPI Scenarios
ax = axes[0, 1]
ax.plot(hist_dates, data.loc[hist_dates, "P_steel_ppi"], 'k-', linewidth=2, label='Historical')
for s in range(n_plot):
    ax.plot(future_dates, P_paths_ppi.iloc[:, s], alpha=0.3, linewidth=0.8, color='green')
ax.plot(future_dates, P_paths_ppi.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
ax.fill_between(future_dates, 
                P_paths_ppi.quantile(0.1, axis=1), 
                P_paths_ppi.quantile(0.9, axis=1),
                alpha=0.2, color='red', label='10th-90th Percentile')
ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_title('Steel Price PPI Projection', fontsize=12, fontweight='bold')
ax.set_ylabel('PPI Index (1982=100)', fontsize=10)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 3: Scrap Cost PPI Scenarios
ax = axes[0, 2]
ax.plot(hist_dates, data.loc[hist_dates, "C_scrap_ppi"], 'k-', linewidth=2, label='Historical')
for s in range(n_plot):
    ax.plot(future_dates, C_paths_ppi.iloc[:, s], alpha=0.3, linewidth=0.8, color='orange')
ax.plot(future_dates, C_paths_ppi.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
ax.fill_between(future_dates, 
                C_paths_ppi.quantile(0.1, axis=1), 
                C_paths_ppi.quantile(0.9, axis=1),
                alpha=0.2, color='red', label='10th-90th Percentile')
ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_title('Scrap Cost PPI Projection', fontsize=12, fontweight='bold')
ax.set_ylabel('PPI Index (1982=100)', fontsize=10)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- ROW 2: Real Values (€/ton) ---

# Plot 4: Demand in Tons (if scaled)
ax = axes[1, 0]
if use_demand_scaling_to_tons:
    # Show historical scaled demand if available
    hist_demand_tons = D0_real * (data.loc[hist_dates, "D_steel_ip"] / D_idx_t0)
    ax.plot(hist_dates, hist_demand_tons, 'k-', linewidth=2, label='Historical')
    
    for s in range(n_plot):
        ax.plot(future_dates, D_final.iloc[:, s], alpha=0.3, linewidth=0.8, color='steelblue')
    ax.plot(future_dates, D_final.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
    ax.fill_between(future_dates, 
                    D_final.quantile(0.1, axis=1), 
                    D_final.quantile(0.9, axis=1),
                    alpha=0.2, color='red', label='10th-90th Percentile')
    ax.set_ylabel('Demand (tons/month)', fontsize=10)
    ax.set_title('Demand Volume Projection', fontsize=12, fontweight='bold')
else:
    # Just show index version again with note
    ax.plot(hist_dates, data.loc[hist_dates, "D_steel_ip"], 'k-', linewidth=2, label='Historical')
    for s in range(n_plot):
        ax.plot(future_dates, D_final.iloc[:, s], alpha=0.3, linewidth=0.8, color='steelblue')
    ax.plot(future_dates, D_final.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
    ax.set_ylabel('Index (1982=100)', fontsize=10)
    ax.set_title('Demand Index Projection\n(Set use_demand_scaling_to_tons=True for tons)', fontsize=12, fontweight='bold')

ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 5: Steel Price in €/ton
ax = axes[1, 1]
# Historical prices (anchored)
hist_price_eur = P0_real * (data.loc[hist_dates, "P_steel_ppi"] / P_ppi_t0)
ax.plot(hist_dates, hist_price_eur, 'k-', linewidth=2, label='Historical')

for s in range(n_plot):
    ax.plot(future_dates, P_paths_eur.iloc[:, s], alpha=0.3, linewidth=0.8, color='green')
ax.plot(future_dates, P_paths_eur.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
ax.fill_between(future_dates, 
                P_paths_eur.quantile(0.1, axis=1), 
                P_paths_eur.quantile(0.9, axis=1),
                alpha=0.2, color='red', label='10th-90th Percentile')
ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_title('Steel Price Projection (Real)', fontsize=12, fontweight='bold')
ax.set_ylabel('Price (€/ton)', fontsize=10)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 6: Scrap Cost in €/ton
ax = axes[1, 2]
hist_cost_eur = C0_real * (data.loc[hist_dates, "C_scrap_ppi"] / C_ppi_t0)
ax.plot(hist_dates, hist_cost_eur, 'k-', linewidth=2, label='Historical')

for s in range(n_plot):
    ax.plot(future_dates, C_paths_eur.iloc[:, s], alpha=0.3, linewidth=0.8, color='orange')
ax.plot(future_dates, C_paths_eur.mean(axis=1), 'r-', linewidth=2, label='Mean Projection')
ax.fill_between(future_dates, 
                C_paths_eur.quantile(0.1, axis=1), 
                C_paths_eur.quantile(0.9, axis=1),
                alpha=0.2, color='red', label='10th-90th Percentile')
ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_title('Scrap Cost Projection (Real)', fontsize=12, fontweight='bold')
ax.set_ylabel('Cost (€/ton)', fontsize=10)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Format all x-axes
for ax in axes.flat:
    ax.set_xlabel('Date', fontsize=10)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*70)
print("SCENARIO PROJECTION SUMMARY")
print("="*70)
print(f"Projection Period: {future_dates[0].strftime('%Y-%m')} to {future_dates[-1].strftime('%Y-%m')}")
print(f"Number of Scenarios: {N_SCENARIOS}")
print(f"Horizon: {HORIZON_MONTHS} months")
print("\n" + "-"*70)

for var_name, paths, unit in [
    ("Demand Index", D_paths_idx, "index"),
    ("Steel Price PPI", P_paths_ppi, "index"),
    ("Scrap Cost PPI", C_paths_ppi, "index"),
    ("Steel Price", P_paths_eur, "€/ton"),
    ("Scrap Cost", C_paths_eur, "€/ton")
]:
    mean_val = paths.mean().mean()
    std_val = paths.std().mean()
    q10 = paths.quantile(0.1, axis=1).mean()
    q90 = paths.quantile(0.9, axis=1).mean()
    
    print(f"{var_name:20s}: {mean_val:8.2f} ± {std_val:6.2f} {unit}")
    print(f"{'':20s}  [10th: {q10:7.2f}, 90th: {q90:7.2f}]")
    print()

In [ ]:
# sim_ret: (S, T, k)
sim_df = pd.DataFrame(
    sim_ret.reshape(-1, sim_ret.shape[2]),
    columns=df_ret.columns
)

qs = np.linspace(0.01, 0.99, 99)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, col in zip(axes, df_ret.columns):
    hq = df_ret[col].quantile(qs).values
    sq = sim_df[col].quantile(qs).values

    ax.plot(hq, sq, marker=".", linestyle="none")
    mn = min(hq.min(), sq.min())
    mx = max(hq.max(), sq.max())
    ax.plot([mn, mx], [mn, mx], linewidth=1)

    ax.set_title(f"QQ (Δlog): {col}")
    ax.set_xlabel("Historical Δlog quantiles")
    ax.set_ylabel("Simulated Δlog quantiles")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

bins = 60
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, col in zip(axes, df_ret.columns):
    ax.hist(df_ret[col].dropna(), bins=bins, density=True, alpha=0.5, label="Historical")
    ax.hist(sim_df[col].dropna(),  bins=bins, density=True, alpha=0.5, label="Simulated")
    ax.set_title(f"Δlog distribution: {col}")
    ax.legend()

plt.tight_layout()
plt.show()

## Scenario Clustering

In [ ]:
# pip install scikit-learn-extra

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn_extra.cluster import KMedoids

def build_feature_matrix(D: pd.DataFrame, P: pd.DataFrame, C: pd.DataFrame):
    # stack paths into one vector per scenario
    scen_names = D.columns
    X = []
    for s in scen_names:
        v = np.concatenate([D[s].values, P[s].values, C[s].values])  # (3T,)
        X.append(v)
    return np.vstack(X), list(scen_names)

def reduce_scenarios_kmedoids(D, P, C, K=30, stress_pct=0.0, seed=42):
    """
    Reduce scenarios using K-Medoids clustering with optional stress scenario inclusion.
    
    Parameters:
    -----------
    D, P, C : pd.DataFrame
        Scenario DataFrames for demand, price, and cost
    K : int
        Number of clusters (representative scenarios)
    stress_pct : float
        Fraction (0-1) of extreme tail scenarios to include for stress testing
        E.g., 0.05 means 5% of K will be reserved for tail scenarios
    seed : int
        Random seed for reproducibility
    
    Returns:
    --------
    D_red, P_red, C_red : pd.DataFrame
        Reduced scenario DataFrames
    w : pd.Series
        Scenario weights (probabilities)
    """
    X, scen_names = build_feature_matrix(D, P, C)

    # Standardize so D/P/C blocks contribute comparably
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    # Calculate number of stress scenarios to include
    n_stress = int(np.ceil(K * stress_pct))
    n_regular = K - n_stress
    
    if n_stress > 0:
        print(f"Including {n_stress} stress scenarios ({stress_pct*100:.1f}%) out of {K} total")
        
        # Step 1: Identify tail scenarios using extreme values
        # Calculate severity score for each scenario (distance from median in standardized space)
        median_scenario = np.median(Xs, axis=0)
        distances = np.linalg.norm(Xs - median_scenario, axis=1)
        
        # Select most extreme scenarios (largest distances)
        extreme_idx = np.argsort(distances)[-n_stress:]
        extreme_scen = [scen_names[i] for i in extreme_idx]
        
        # Step 2: Cluster remaining scenarios
        remaining_idx = np.setdiff1d(np.arange(len(scen_names)), extreme_idx)
        Xs_regular = Xs[remaining_idx]
        scen_names_regular = [scen_names[i] for i in remaining_idx]
        
        if n_regular > 0:
            km = KMedoids(n_clusters=n_regular, random_state=seed, method="pam")
            labels_regular = km.fit_predict(Xs_regular)
            
            medoid_idx_regular = km.medoid_indices_
            rep_scen_regular = [scen_names_regular[i] for i in medoid_idx_regular]
            
            # Calculate weights for regular scenarios
            weights_regular = pd.Series(labels_regular).value_counts().sort_index()
            total_regular_prob = 1.0 - stress_pct
            prob_regular = (weights_regular / weights_regular.sum() * total_regular_prob).to_dict()
        else:
            rep_scen_regular = []
            prob_regular = {}
        
        # Combine regular and stress scenarios
        rep_scen = rep_scen_regular + extreme_scen
        
        # Assign equal weight to each stress scenario
        stress_weight = stress_pct / n_stress if n_stress > 0 else 0.0
        
        # Build combined probability dictionary
        rep_prob = {}
        for j, scen in enumerate(rep_scen):
            if scen in extreme_scen:
                rep_prob[scen] = float(stress_weight)
            else:
                cluster_label = labels_regular[scen_names_regular.index(scen)]
                rep_prob[scen] = float(prob_regular[cluster_label])
    
    else:
        # Original behavior: no stress scenarios
        km = KMedoids(n_clusters=K, random_state=seed, method="pam")
        labels = km.fit_predict(Xs)
        
        medoid_idx = km.medoid_indices_
        rep_scen = [scen_names[i] for i in medoid_idx]
        
        weights = pd.Series(labels).value_counts().sort_index()
        prob = (weights / weights.sum()).to_dict()
        
        rep_prob = {}
        for j, scen in enumerate(rep_scen):
            rep_prob[scen] = float(prob[j])

    # return reduced scenario paths + weights
    D_red = D[rep_scen].copy()
    P_red = P[rep_scen].copy()
    C_red = C[rep_scen].copy()

    w = pd.Series(rep_prob, name="prob").sort_index()
    return D_red, P_red, C_red, w

# Example usage:
# stress_pct=0.0 → original behavior (no stress scenarios)
# stress_pct=0.05 → 5% of scenarios are extreme tail scenarios for stress testing
D_red, P_red, C_red, w = reduce_scenarios_kmedoids(
    D_final, 
    P_paths_eur, 
    C_paths_eur, 
    K=1_000,
    stress_pct=0.01)  # Include 1% stress scenarios

print(f"\nTotal scenarios: {len(w)}")
print(f"Total probability: {w.sum():.4f}")
print(f"\nWeight statistics:")
print(f"  Min: {w.min():.4f}")
print(f"  Max: {w.max():.4f}")
print(f"  Mean: {w.mean():.4f}")
print(f"  Std: {w.std():.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ensure w is aligned with D_red.columns
w = w.reindex(D_red.columns)

# Function to compute weighted quantile
def weighted_quantile(values, weights, q):
    sorter = np.argsort(values)
    values = values[sorter]
    weights = weights[sorter]
    cum_weights = np.cumsum(weights)
    total_weight = cum_weights[-1]
    idx = np.searchsorted(cum_weights, q * total_weight)
    if idx == 0:
        return values[0]
    elif idx >= len(values):
        return values[-1]
    else:
        return values[idx-1] + (values[idx] - values[idx-1]) * (q * total_weight - cum_weights[idx-1]) / weights[idx]

# Generate date range for plotting
start_date = (data.index.max() + pd.offsets.MonthBegin(1)).normalize()
future_dates = pd.date_range(start=start_date, periods=HORIZON_MONTHS, freq="MS")

hist_months = 32
hist_dates = data.index[-hist_months:]

# Normalize weights for linewidth scaling (min 0.5, max 3)
w_min, w_max = w.min(), w.max()
def scale_weight(weight):
    return 0.5 + 2.5 * (weight - w_min) / (w_max - w_min + 1e-9)

# Create plots for reduced scenarios (1 row, 3 columns)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Helper function to plot scenarios
def plot_reduced_scenarios(ax, hist_data, future_data, weights, color, title, ylabel):
    # Historical
    ax.plot(hist_dates, hist_data, 'k-', linewidth=2, label='Historical')
    
    # Plot each scenario with linewidth proportional to weight (no individual labels)
    for s in future_data.columns:
        lw = scale_weight(weights[s])
        ax.plot(future_dates, future_data[s], alpha=0.5, linewidth=lw, color=color)
    
    # Weighted mean
    weighted_mean = (future_data * weights).sum(axis=1)
    ax.plot(future_dates, weighted_mean, 'r-', linewidth=2, label='Weighted Mean')
    
    # Weighted quantiles
    q10 = [weighted_quantile(future_data.iloc[t, :].values, weights.values, 0.1) for t in range(len(future_dates))]
    q90 = [weighted_quantile(future_data.iloc[t, :].values, weights.values, 0.9) for t in range(len(future_dates))]
    ax.fill_between(future_dates, q10, q90, alpha=0.2, color='red', label='10th-90th Percentile')
    
    ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=10)
    ax.legend(fontsize=8, loc='upper left')
    ax.grid(True, alpha=0.3)

# Plot 1: Demand
hist_demand = D0_real * (data.loc[hist_dates, "D_steel_ip"] / D_idx_t0) if use_demand_scaling_to_tons else data.loc[hist_dates, "D_steel_ip"]
ylabel_d = 'Demand (tons/month)' if use_demand_scaling_to_tons else 'Index'
plot_reduced_scenarios(axes[0], hist_demand, D_red, w, 'steelblue', 'Reduced Demand Projection', ylabel_d)

# Plot 2: Steel Price
hist_price = P0_real * (data.loc[hist_dates, "P_steel_ppi"] / P_ppi_t0)
plot_reduced_scenarios(axes[1], hist_price, P_red, w, 'green', 'Reduced Steel Price Projection', 'Price (€/ton)')

# Plot 3: Scrap Cost
hist_cost = C0_real * (data.loc[hist_dates, "C_scrap_ppi"] / C_ppi_t0)
plot_reduced_scenarios(axes[2], hist_cost, C_red, w, 'orange', 'Reduced Scrap Cost Projection', 'Cost (€/ton)')

# Format x-axes
for ax in axes:
    ax.set_xlabel('Date', fontsize=10)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*70)
print("REDUCED SCENARIO PROJECTION SUMMARY")
print("="*70)
print(f"Number of Reduced Scenarios: {len(D_red.columns)}")
print(f"Total Weight: {w.sum():.3f}")
print(f"Weight range: [{w.min():.3f}, {w.max():.3f}]")
print("Line thickness represents scenario probability (thicker = higher weight)")

# Stochastic Program

In [ ]:
import pandas as pd
import pyomo.environ as pyo

def build_sp_model(
    scenarios: pd.DataFrame,
    prob: pd.Series,
    alpha: float,
    c_var: float,
    c_cap_base: float,
    c_cap_flex: float,
    delta_base: float,
    delta_spot: float,
    pen_unmet: float,
    gamma_cap: float,
    gamma_scrap: float,
):
    # -----------------------------
    # Clean / align inputs
    # -----------------------------
    df = scenarios.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df["Scenario"] = df["Scenario"].astype(str)

    prob = prob.copy()
    prob.index = prob.index.astype(str)

    # Validate probability mass
    psum = float(prob.sum())
    if abs(psum - 1.0) > 1e-6:
        raise ValueError(f"Scenario probabilities must sum to 1. Got {psum}")

    T = sorted(df["Date"].unique())
    S = sorted(df["Scenario"].unique())

    missing_probs = [s for s in S if s not in prob.index]
    if missing_probs:
        raise ValueError(f"Missing probabilities for scenarios: {missing_probs[:10]}")

    # Build parameter dicts
    # (t,s) -> value
    D = {}
    P = {}
    C = {}

    # Fast lookup via multi-index
    df2 = df.set_index(["Date", "Scenario"]).sort_index()

    for t in T:
        for s in S:
            try:
                row = df2.loc[(t, s)]
            except KeyError as e:
                raise ValueError(f"Missing (Date,Scenario)=({t},{s}) in scenarios_df") from e

            D[(t, s)] = float(row["D"])
            P[(t, s)] = float(row["P"])
            C[(t, s)] = float(row["C"])

    p = {s: float(prob.loc[s]) for s in S}

    # -----------------------------
    # Build model
    # -----------------------------
    m = pyo.ConcreteModel()

    m.T = pyo.Set(initialize=T, ordered=True)
    m.S = pyo.Set(initialize=S, ordered=True)

    # Stage 1
    m.Cap_base = pyo.Var(m.T, domain=pyo.NonNegativeReals)
    m.Q_base   = pyo.Var(m.T, domain=pyo.NonNegativeReals)

    # Stage 2 (recourse)
    m.Cap_flex = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)
    m.x        = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)  # production
    m.y        = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)  # sales
    m.u        = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)  # unmet demand
    m.q_base   = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)  # called from base
    m.q_spot   = pyo.Var(m.T, m.S, domain=pyo.NonNegativeReals)  # spot bought

    # -----------------------------
    # Constraints
    # -----------------------------
    # Demand balance: y + u = D
    def demand_bal(m, t, s):
        return m.y[t, s] + m.u[t, s] == D[(t, s)]
    m.DemandBal = pyo.Constraint(m.T, m.S, rule=demand_bal)

    # No finished-goods inventory: y <= x
    def no_fg_inv(m, t, s):
        return m.y[t, s] <= m.x[t, s]
    m.NoFGInv = pyo.Constraint(m.T, m.S, rule=no_fg_inv)

    # Capacity: x <= Cap_base + Cap_flex
    def cap_link(m, t, s):
        return m.x[t, s] <= m.Cap_base[t] + m.Cap_flex[t, s]
    m.CapLink = pyo.Constraint(m.T, m.S, rule=cap_link)

    # Scrap consumption: alpha*x = q_base + q_spot
    def scrap_bal(m, t, s):
        return alpha * m.x[t, s] == m.q_base[t, s] + m.q_spot[t, s]
    m.ScrapBal = pyo.Constraint(m.T, m.S, rule=scrap_bal)

    # Base call limit: q_base <= Q_base
    def base_call_limit(m, t, s):
        return m.q_base[t, s] <= m.Q_base[t]
    m.BaseCallLimit = pyo.Constraint(m.T, m.S, rule=base_call_limit)

    # Recourse bound for capacity: Cap_flex <= gamma_cap * Cap_base
    def flex_cap_bound(m, t, s):
        return m.Cap_flex[t, s] <= gamma_cap * m.Cap_base[t]
    m.FlexCapBound = pyo.Constraint(m.T, m.S, rule=flex_cap_bound)

    # Recourse bound for scrap: q_spot <= gamma_scrap * Q_base
    def spot_scrap_bound(m, t, s):
        return m.q_spot[t, s] <= gamma_scrap * m.Q_base[t]
    m.SpotScrapBound = pyo.Constraint(m.T, m.S, rule=spot_scrap_bound)

    # -----------------------------
    # Objective: maximize expected profit
    # -----------------------------
    def obj_rule(m):
        # Expected scenario profit
        exp_profit = 0.0
        for s in m.S:
            scen_profit = 0.0
            for t in m.T:
                scen_profit += (
                    P[(t, s)] * m.y[t, s]
                    - C[(t, s)] * (m.q_base[t, s] + m.q_spot[t, s])
                    - c_var * m.x[t, s]
                    - c_cap_flex * m.Cap_flex[t, s]
                    - delta_spot * m.q_spot[t, s]
                    - pen_unmet * m.u[t, s]
                )
            exp_profit += p[s] * scen_profit

        # Stage-1 costs (not scenario-weighted)
        stage1_cost = sum(
            c_cap_base * m.Cap_base[t] + delta_base * m.Q_base[t]
            for t in m.T
        )

        return exp_profit - stage1_cost

    m.Obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

    return m

# -----------------------------
# Solve helper
# -----------------------------
def solve_sp(model, solver_name="gurobi", tee=True):
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' is not available.")
    res = solver.solve(model, tee=tee)
    return res

# -----------------------------
# Results extraction helper
# -----------------------------
def extract_results(model):
    cap_base = pd.Series({t: pyo.value(model.Cap_base[t]) for t in model.T}).sort_index()
    q_base   = pd.Series({t: pyo.value(model.Q_base[t]) for t in model.T}).sort_index()

    # Expected totals across scenarios
    # (weights are embedded in your prob input; if you want them here, pass prob separately)
    return {
        "Cap_base": cap_base,
        "Q_base": q_base,
    }

scenarios_red = scenarios[scenarios["Scenario"].isin(w.index.values)].copy()
scenarios_red = scenarios_red.rename(columns={'D_steel_ip': 'D', 'P_steel_ppi': 'P', 'C_scrap_ppi': 'C'})

m = build_sp_model(
    scenarios=scenarios_red,
    prob=w,
    alpha=1.0, # scrap-to-product ratio
    c_var=200.0, # variable production cost €/ton
    c_cap_base=5.0, # base capacity cost €/ton-month
    c_cap_flex=15.0, # flex capacity cost €/ton-month
    delta_base=5.0, # cost to call base scrap €/ton
    delta_spot=20.0, # cost to buy spot scrap €/ton
    pen_unmet=200.0, # penalty for unmet demand €/ton
    gamma_cap=0.3, # max flex capacity as fraction of base
    gamma_scrap=1.0, # max spot scrap as fraction of base call
)

res = solve_sp(m, solver_name="highs", tee=True)
print(res.solver.termination_condition)
out = extract_results(m)
print("\nOptimal Base Capacity (tons/month):")
print(out["Cap_base"])
print("\nOptimal Base Call (tons/month):")
print(out["Q_base"])

In [ ]:
def analyze_profit_distribution(model, prob, confidence_levels=[0.05, 0.95]):
    """
    Analyze profit distribution across scenarios and compute confidence intervals.
    
    Parameters:
    -----------
    model : pyomo model (solved)
    prob : pd.Series with scenario probabilities
    confidence_levels : list of quantile levels for CI
    
    Returns:
    --------
    dict with profit statistics
    """
    import numpy as np
    
    # Extract scenario-specific profits
    scenario_profits = []
    scenario_names = []
    weights = []
    
    for s in model.S:
        profit_s = 0.0
        for t in model.T:
            # Get parameter values for this (t,s)
            D_ts = sum(1 for t2, s2 in model.DemandBal if t2 == t and s2 == s)  # This won't work
            # We need to rebuild the parameter dictionaries or store them
    
    # Since we need the original parameters, let's rebuild them
    scenarios_red = scenarios[scenarios["Scenario"].isin(prob.index.values)].copy()
    scenarios_red = scenarios_red.rename(columns={'D_steel_ip': 'D', 'P_steel_ppi': 'P', 'C_scrap_ppi': 'C'})
    
    df = scenarios_red.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df["Scenario"] = df["Scenario"].astype(str)
    df2 = df.set_index(["Date", "Scenario"]).sort_index()
    
    T = sorted(df["Date"].unique())
    S = sorted(df["Scenario"].unique())
    
    # Rebuild parameter dicts
    D = {}
    P = {}
    C = {}
    for t in T:
        for s in S:
            row = df2.loc[(t, s)]
            D[(t, s)] = float(row["D"])
            P[(t, s)] = float(row["P"])
            C[(t, s)] = float(row["C"])
    
    # Extract model parameters
    alpha = 1.0
    c_var = 200.0
    c_cap_flex = 15.0
    delta_spot = 20.0
    pen_unmet = 200.0
    
    # Calculate profit for each scenario
    for s in S:
        profit_s = 0.0
        for t in T:
            profit_s += (
                P[(t, s)] * pyo.value(model.y[t, s])
                - C[(t, s)] * (pyo.value(model.q_base[t, s]) + pyo.value(model.q_spot[t, s]))
                - c_var * pyo.value(model.x[t, s])
                - c_cap_flex * pyo.value(model.Cap_flex[t, s])
                - delta_spot * pyo.value(model.q_spot[t, s])
                - pen_unmet * pyo.value(model.u[t, s])
            )
        
        scenario_profits.append(profit_s)
        scenario_names.append(s)
        weights.append(float(prob.loc[s]))
    
    scenario_profits = np.array(scenario_profits)
    weights = np.array(weights)
    
    # Calculate expected profit (weighted mean)
    expected_profit = np.average(scenario_profits, weights=weights)
    
    # Calculate weighted percentiles for confidence intervals
    def weighted_quantile(values, weights, q):
        sorter = np.argsort(values)
        values = values[sorter]
        weights = weights[sorter]
        cum_weights = np.cumsum(weights)
        total_weight = cum_weights[-1]
        idx = np.searchsorted(cum_weights, q * total_weight)
        if idx == 0:
            return values[0]
        elif idx >= len(values):
            return values[-1]
        else:
            return values[idx-1] + (values[idx] - values[idx-1]) * (q * total_weight - cum_weights[idx-1]) / weights[idx]
    
    # Calculate confidence intervals
    ci_lower = weighted_quantile(scenario_profits, weights, confidence_levels[0])
    ci_upper = weighted_quantile(scenario_profits, weights, confidence_levels[1])
    
    # Calculate additional statistics
    profit_std = np.sqrt(np.average((scenario_profits - expected_profit)**2, weights=weights))
    profit_min = np.min(scenario_profits)
    profit_max = np.max(scenario_profits)
    
    return {
        'expected_profit': expected_profit,
        'profit_std': profit_std,
        'profit_min': profit_min,
        'profit_max': profit_max,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'confidence_level': f"{(confidence_levels[1] - confidence_levels[0])*100:.0f}%",
        'scenario_profits': pd.Series(scenario_profits, index=scenario_names),
        'scenario_weights': pd.Series(weights, index=scenario_names)
    }

# Analyze profit distribution
profit_stats = analyze_profit_distribution(m, w)

print("\n" + "="*60)
print("PROFIT DISTRIBUTION ANALYSIS")
print("="*60)
print(f"Expected Profit: €{profit_stats['expected_profit']:,.0f}")
print(f"Standard Deviation: €{profit_stats['profit_std']:,.0f}")
print(f"Minimum Profit: €{profit_stats['profit_min']:,.0f}")
print(f"Maximum Profit: €{profit_stats['profit_max']:,.0f}")
print(f"\n{profit_stats['confidence_level']} Confidence Interval:")
print(f"  Lower bound: €{profit_stats['ci_lower']:,.0f}")
print(f"  Upper bound: €{profit_stats['ci_upper']:,.0f}")

# Plot profit distribution
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of scenario profits
scenario_profits = profit_stats['scenario_profits']
weights = profit_stats['scenario_weights']

ax1.hist(scenario_profits, bins=20, weights=weights, alpha=0.7, edgecolor='black')
ax1.axvline(profit_stats['expected_profit'], color='red', linestyle='--', linewidth=2, label='Expected Profit')
ax1.axvline(profit_stats['ci_lower'], color='orange', linestyle='--', linewidth=1.5, label=f"{profit_stats['confidence_level']} CI")
ax1.axvline(profit_stats['ci_upper'], color='orange', linestyle='--', linewidth=1.5)
ax1.set_xlabel('Profit (€)')
ax1.set_ylabel('Probability Density')
ax1.set_title('Weighted Profit Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Scatter plot: profit vs scenario weight
ax2.scatter(weights, scenario_profits, alpha=0.7, s=50)
ax2.axhline(profit_stats['expected_profit'], color='red', linestyle='--', linewidth=2, label='Expected Profit')
ax2.axhline(profit_stats['ci_lower'], color='orange', linestyle='--', linewidth=1.5, label=f"{profit_stats['confidence_level']} CI")
ax2.axhline(profit_stats['ci_upper'], color='orange', linestyle='--', linewidth=1.5)
ax2.set_xlabel('Scenario Weight')
ax2.set_ylabel('Profit (€)')
ax2.set_title('Profit vs Scenario Probability')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show worst and best case scenarios
print(f"\nWorst Case Scenario (ID: {scenario_profits.idxmin()}):")
print(f"  Profit: €{scenario_profits.min():,.0f}")
print(f"  Weight: {weights[scenario_profits.idxmin()]:.4f}")

print(f"\nBest Case Scenario (ID: {scenario_profits.idxmax()}):")
print(f"  Profit: €{scenario_profits.max():,.0f}")
print(f"  Weight: {weights[scenario_profits.idxmax()]:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Extract scenario profits from the analysis
scenario_profits = profit_stats['scenario_profits']

# Create a colormap for profits
norm = Normalize(vmin=scenario_profits.min(), vmax=scenario_profits.max())
cmap = cm.RdYlGn  # Red-Yellow-Green colormap (red=low profit, green=high profit)

# Generate date range for plotting
start_date = (data.index.max() + pd.offsets.MonthBegin(1)).normalize()
future_dates = pd.date_range(start=start_date, periods=HORIZON_MONTHS, freq="MS")

hist_months = 60
hist_dates = data.index[-hist_months:]

# Create plots with profit-colored scenarios
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

def plot_profit_colored_scenarios(ax, hist_data, future_data, scenario_profits, title, ylabel):
    # Historical data
    ax.plot(hist_dates, hist_data, 'k-', linewidth=3, label='Historical', alpha=0.8)
    
    # Plot each scenario colored by profit
    for s in future_data.columns:
        if s in scenario_profits.index:
            profit = scenario_profits[s]
            color = cmap(norm(profit))
            alpha = 0.6
            linewidth = 1.5
        else:
            color = 'gray'
            alpha = 0.3
            linewidth = 0.5
        
        ax.plot(future_dates, future_data[s], color=color, alpha=alpha, linewidth=linewidth)
    
    # Add vertical line at transition
    ax.axvline(x=data.index.max(), color='gray', linestyle='--', linewidth=2, alpha=0.8)
    
    # Formatting
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_xlabel('Date', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

# Plot 1: Demand colored by profit
hist_demand = D0_real * (data.loc[hist_dates, "D_steel_ip"] / D_idx_t0) if use_demand_scaling_to_tons else data.loc[hist_dates, "D_steel_ip"]
ylabel_d = 'Demand (tons/month)' if use_demand_scaling_to_tons else 'Index'
plot_profit_colored_scenarios(
    axes[0], hist_demand, D_red, scenario_profits,
    'Demand Scenarios\n(Colored by Profit)', ylabel_d
)

# Plot 2: Steel Price colored by profit
hist_price = P0_real * (data.loc[hist_dates, "P_steel_ppi"] / P_ppi_t0)
plot_profit_colored_scenarios(
    axes[1], hist_price, P_red, scenario_profits,
    'Steel Price Scenarios\n(Colored by Profit)', 'Price (€/ton)'
)

# Plot 3: Scrap Cost colored by profit
hist_cost = C0_real * (data.loc[hist_dates, "C_scrap_ppi"] / C_ppi_t0)
plot_profit_colored_scenarios(
    axes[2], hist_cost, C_red, scenario_profits,
    'Scrap Cost Scenarios\n(Colored by Profit)', 'Cost (€/ton)'
)

# Format x-axes
for ax in axes:
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

# Apply tight_layout first to position the plots
plt.tight_layout()

# Then add colorbar with manual positioning
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
# Use figure.add_axes to manually position the colorbar
cbar_ax = fig.add_axes([0.15, 0, 0.7, 0.03])  # [left, bottom, width, height]
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Scenario Profit (€)', fontsize=12, fontweight='bold')

plt.show()

# Print insights about profit vs scenario characteristics
print("\n" + "="*80)
print("PROFIT vs SCENARIO CHARACTERISTICS ANALYSIS")
print("="*80)

# Analyze correlation between final period values and profits
final_period_data = {}
for s in scenario_profits.index:
    if s in D_red.columns:
        final_period_data[s] = {
            'demand': D_red[s].iloc[-1],
            'price': P_red[s].iloc[-1], 
            'cost': C_red[s].iloc[-1],
            'profit': scenario_profits[s]
        }

final_df = pd.DataFrame(final_period_data).T

# Calculate correlations
correlations = final_df.corr()['profit'].drop('profit')
print("Correlation between final period values and total profit:")
for var, corr in correlations.items():
    print(f"  {var.capitalize():8s}: {corr:+.3f}")

# Show top and bottom scenarios
print(f"\nTop 3 Most Profitable Scenarios:")
top_scenarios = scenario_profits.nlargest(3)
for i, (scen, profit) in enumerate(top_scenarios.items(), 1):
    if scen in final_df.index:
        row = final_df.loc[scen]
        print(f"  {i}. Scenario {scen}: €{profit:,.0f}")
        print(f"     Final values - Demand: {row['demand']:,.0f}, Price: €{row['price']:.0f}, Cost: €{row['cost']:.0f}")

print(f"\nTop 3 Least Profitable Scenarios:")
bottom_scenarios = scenario_profits.nsmallest(3)
for i, (scen, profit) in enumerate(bottom_scenarios.items(), 1):
    if scen in final_df.index:
        row = final_df.loc[scen]
        print(f"  {i}. Scenario {scen}: €{profit:,.0f}")
        print(f"     Final values - Demand: {row['demand']:,.0f}, Price: €{row['price']:.0f}, Cost: €{row['cost']:.0f}")

print(f"\nKey Insights:")
print(f"- Green lines = High profit scenarios")
print(f"- Red lines = Low profit scenarios") 
print(f"- Most profitable scenarios tend to have: {correlations.idxmax()} correlation")
print(f"- Least profitable scenarios tend to have: {correlations.idxmin()} correlation")